<a href="https://colab.research.google.com/github/gnbucketz/pynqmnistnumberreader/blob/main/trainingmodelpynqz2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() #input and output

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
x_train = x_train.reshape((60000, 28, 28, 1))
x_train= x_train/ 255.0
x_test = x_test.reshape((10000, 28, 28, 1))
x_test= x_test/ 255.0

In [ ]:
def CNN():
    inputs = keras.Input(shape=(28, 28, 1), name='Input_layer')
    x = layers.Conv2D(filters=32, kernel_size=3, strides=(1, 1), padding='valid', activation='relu', name="conv_layer_1")(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2), name="pooling_1")(x)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name="conv_layer_2")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), name="pooling_2")(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', name="conv_layer_3")(x)
    x = layers.Flatten(name="flattening_layer")(x)
    x = layers.Dense(units=64, activation='relu')(x)
    outputs = layers.Dense(units=10, activation='softmax', name='output_layer')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='final_CNN')
    model.compile(optimizer='rmsprop',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model = CNN()
model.summary()

Model: "final_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 conv_layer_1 (Conv2D)       (None, 26, 26, 32)        320       
                                                                 
 pooling_1 (MaxPooling2D)    (None, 13, 13, 32)        0         
                                                                 
 conv_layer_2 (Conv2D)       (None, 11, 11, 64)        18496     
                                                                 
 pooling_2 (MaxPooling2D)    (None, 5, 5, 64)          0         
                                                                 
 conv_layer_3 (Conv2D)       (None, 3, 3, 128)         73856     
                                                                 
 flattening_layer (Flatten)  (None, 1152)              0 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
modelcheckpoint = ModelCheckpoint(filepath="final_CNN.h5", save_best_only=True, monitor="val_loss")

In [ ]:
history = model.fit(x=x_train, y=y_train,
                    validation_data=(x_test, y_test),
                    epochs=10, batch_size=64, callbacks=[modelcheckpoint])

Epoch 1/10
938/938 [==============================] - 73s 77ms/step - loss: 0.1651 - accuracy: 0.9480 - val_loss: 0.0464 - val_accuracy: 0.9846
Epoch 2/10
  3/938 [..............................] - ETA: 40s - loss: 0.0272 - accuracy: 0.9896

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


938/938 [==============================] - 51s 54ms/step - loss: 0.0437 - accuracy: 0.9867 - val_loss: 0.0485 - val_accuracy: 0.9832
Epoch 3/10
938/938 [==============================] - 51s 55ms/step - loss: 0.0313 - accuracy: 0.9908 - val_loss: 0.0282 - val_accuracy: 0.9915
Epoch 4/10
938/938 [==============================] - 54s 57ms/step - loss: 0.0219 - accuracy: 0.9933 - val_loss: 0.0511 - val_accuracy: 0.9837
Epoch 5/10
938/938 [==============================] - 52s 55ms/step - loss: 0.0174 - accuracy: 0.9947 - val_loss: 0.0350 - val_accuracy: 0.9899
Epoch 6/10
938/938 [==============================] - 53s 56ms/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.0351 - val_accuracy: 0.9906
Epoch 7/10
938/938 [==============================] - 51s 55ms/step - loss: 0.0112 - accuracy: 0.9966 - val_loss: 0.0288 - val_accuracy: 0.9908
Epoch 8/10
938/938 [==============================] - 51s 55ms/step - loss: 0.0087 - accuracy: 0.9976 - val_loss: 0.0335 - val_accuracy: 0.9913
Epo

In [ ]:
import tensorflow as tf

In [ ]:
# Load your Keras model
model = tf.keras.models.load_model('final_CNN.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('cnn_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
def convert_tflite_to_c_array(tflite_file_path, output_header_file_path):
    with open(tflite_file_path, 'rb') as f:
        tflite_model = f.read()

    with open(output_header_file_path, 'w') as f:
        f.write('unsigned char g_cnn_model_tflite[] = {')
        for i, byte in enumerate(tflite_model):
            if i % 12 == 0:
                f.write('\n    ')
            f.write(f'0x{byte:02x}, ')
        f.write('\n};\n')
        f.write(f'unsigned int g_cnn_model_tflite_len = {len(tflite_model)};\n')

# Call the function to create the header file
convert_tflite_to_c_array('cnn_model.tflite', 'cnn_model_tflite.h')

In [ ]:
!ls -la /content/


total 6900
drwxr-xr-x 1 root root    4096 May 18 09:57 .
drwxr-xr-x 1 root root    4096 May 18 08:14 ..
-rw-r--r-- 1 root root  671892 May 18 09:33 cnn_model.onnx
-rw-r--r-- 1 root root  672424 May 18 09:53 cnn_model.tflite
-rw-r--r-- 1 root root 4314812 May 18 09:59 cnn_model_tflite.h
drwxr-xr-x 4 root root    4096 May 16 13:24 .config
-rw-r--r-- 1 root root 1380328 May 18 09:21 final_CNN.h5
drwxr-xr-x 1 root root    4096 May 16 13:24 sample_data
